Let's start by importing RTS-96 data:

In [ ]:
####### IMPORTS: DATA ACQUISITION ########
include("../src/tmp_inst_rts96.jl")

psData = psDataLoad()

# unpack psDL (boilerplate):
(Sb,f,t,r,x,b,Y,bustype,
Gp,Gq,Dp,Dq,Rp,Rq,
Pmax,Pmin,Qmax,Qmin,Plim,
Vg,Vceiling,Vfloor,
busIdx,N,Nr,Ng,k) = unpack_psDL(psData)

####### IMPORTS: INSTANTON CODE ########
# Get instanton code into workspace:
include("../src/TemporalInstanton.jl")
include("../src/TrustRegionSubproblem.jl")

using TemporalInstanton, TrustRegionSubproblem

####### DATA LINKING ########
# Static
Ridx = find(Rp) # Vector of renewable nodes
Y = full(Y) # Full admittance matrix (ref not removed)
ref = 1 # Index of ref node
k = k # Conventional generator participation factors
tau = 1. # Heating constant
G0 = [Gp,Gp.*0.95,Gp.*0.9]
D0 = [Dp,Dp.*0.95,Dp.*0.9]
P0 = [Rp,Rp,Rp]
kQobj = 0 # Should always be zero; objective has no constant term
c = 10. # RHS of line temperature constraint
kQtheta = -c # Move to LHS so we have Q(z) = 0

T = int(length(G0)/size(Y,1))

# Changing:
lines = [(f[i],t[i]) for i in 1:length(f)];

# For a single line:
line = lines[48]

# Create instance of instanton problem
Qobj,A,b,Qtheta = temporalInstanton(Ridx,Y,ref,k,tau,line,G0,P0,D0)

In [ ]:
# Build obj and constraint quadratics:
G_of_x = (Qobj,0,kQobj)
Q_of_x = (Qtheta,0,kQtheta)

Ian's most recent set of notes partitions the vector of variables $x$ into three parts:

* $x_1\in\mathbb{R}^{N_rT}$ contains wind deviations

* $x_2\in\mathbb{R}^{(N+1)T}$ contains angle and mismatch variables

* $x_3\in\mathbb{R}^T$ contains angle difference variables involved in line temperature calculation

With this notation, the problem becomes

\begin{align}
&& \min~ & x_1^\top Q_x x_1 \\
s.t. && Ax &= b \\
&& x_3^\top x_3 &= c
\end{align}

where $x=[x_1~x_2~x_3]^\top$.

## Translation

As always, the first step in solving our problem is translation. We need to go from $Ax=b$ to $Ay=0$, where $y=x-x^*$ and $x^*$ is the translation. It is easy to find and translate by a point in the set $\{x:Ax=b\}$, but we have an additional requirement: We don't want to introduce any linear term to the norm constraint.

\begin{align}
&& \min~ & x_3^\top x_3 \\
s.t. && A_1x_1 + A_2x_2 + A_3x_3 &= b
\end{align}

How do we partition $A$?

* $A_1$ contains columns of $A$ corresponding to wind deviations. The indices of these columns are indices for which $Q_{obj}$ is nonero.

* $A_2$ contains columns corresponding to angles and mismatches. These are all the columns remaining after $A_1$ and $A_3$ are removed.

* $A_3$ contains columns corresponding to angle difference variables. This is just the last $T$ columns of $A$.

This function partitions $A$:

In [ ]:
function partition_A(A,Qobj,T)
    """ Return A1, A2, A3 where
    A1 corresponds to wind
    A2 corresponds to angles + mismatch
    A3 corresponds to angle difference vars
    """
    m,n = size(A)
    idx1 = find(diag(Qobj))
    idx2 = setdiff(1:n-T,idx1)
    idx3 = n-T+1:n
    idx = [idx1,idx2,idx3]
    
    A1 = A[:,idx1]
    A2 = A[:,idx2]
    A3 = A[:,idx3]
    return A1,A2,A3,idx1,idx2,idx3
end

Now that $A$ is partitioned, we can find $x_1^*$, $x_2^*$, and $x_3^*$. To avoid messing up the norm, we need $x_3^*=0$. With that constraint satisfied, we just need $A_1x_2 + A_2x_2 = b$, so let's use `[A1 A2]\b` to obtain min-norm $x_1^*$ and $x_2^*$.

In [ ]:
function find_x_star(Qobj,A,b,T)
    A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)
    x_star = zeros(size(A,2))
    x_star[[idx1,idx2]] = [A1 A2]\b
    return x_star
end

The translation itself is a change of variables from $x$ to $y = x - x^*$. The translated problem is

\begin{align}
&& \min~ & y_1^\top Q_x y_1 + 2 y_1^\top x_1^* \\
s.t. && Ay &= 0 \\
&& y_3^\top y_3 &= c
\end{align}

In [ ]:
# Partition A;
A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)

# Find translation point:
x_star = find_x_star(Qobj,A,b,T)

# Translate quadratics:
G_of_y = translate_quadratic(G_of_x,x_star)
Q_of_y = translate_quadratic(Q_of_x,x_star)

## Kernel "rotation"

Now that the problem has been translated, we replace $y$ by $Nz$, where $N=[v_1~v_2~\ldots~v_k]\in\mathbb{R}^{n\times k}$, which will reduce the problem dimension from $n$ variables to $k$ variables.

From Ian's notes:

\begin{align}
\begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix}
&= \begin{bmatrix} N_1 \\ N_2 \\ N_3 \end{bmatrix} z
\end{align}

* The objective becomes $\min z^\top \hat{Q}_{obj} z + 2z^\top N_1x_1^*$, where $\hat{Q}_{obj} = N_1^\top Q N_1$

* The temperature constraint becomes $z^\top N_3^\top N_3 z = c$.

The following method rotates a quadratic:

In [ ]:
function rotate_quadratic(G_of_x,R)
    """ Rotate quadratic G_of_x by
    rotation matrix R.
    """
    G,g,kg = G_of_x
    return (R*G*R',R*g,kg)
end

Here we find and partition $N$, then rotate our objective and constraint quadratics by it:

In [ ]:
N = kernel_rotation(A)[:,1:size(A,2) - rank(A)] # take only first k cols

N1,N2,N3 = N[idx1,:],N[idx2,:],N[idx3,:] # partition N

G_of_z = rotate_quadratic(G_of_y,N')
Q_of_z = rotate_quadratic(Q_of_y,N')

## Constraint Eigendecomposition

Perform an Eigendecomposition: $$N_3^\top N_3 = UDU^\top$$

In [ ]:
D,U = eig(Q_of_z[1])
D = round(D,10)

Now let $\hat{z} = U^\top z$ so that

\begin{align}
z^\top N_3^\top N_3 z &= \hat{z}^\top D\hat{z}
\end{align}

$D$ has at most $T$ nonzero elements, because rank$(N
_3) \leq T$ by virtue of its dimension. It will look like this:

\begin{align}
D &= \begin{bmatrix} 0 & 0 \\ 0 & \hat{D} \end{bmatrix}
\end{align}

Now let $w = [w_1~w_2]^\top$ and $\hat{z}=[\hat{z}_1~\hat{z}_2]^\top$, and relate them as follows:

\begin{align}
\begin{bmatrix} w_1 \\ w_2 \end{bmatrix} &=
\begin{bmatrix} I & 0 \\ 0 & \hat{D}^{1/2} \end{bmatrix}
\begin{bmatrix} \hat{z}_1 \\ \hat{z}_2 \end{bmatrix} \\
\implies w &= K\hat{z} = KU^\top z
\end{align}

In [ ]:
function return_K(D)
    K = ones(length(D))
    K[find(D)] = sqrt(D[find(D)])
    K = diagm(K)
    return K
end

Then

\begin{align}
\hat{z}^\top D\hat{z} &= \hat{z}_1\hat{D}^{1/2}\hat{D}^{1/2}\hat{z} \\
&= w_1^\top w_1
\end{align}

Note that $z = UK^{-1}w$ because $UU^\top = I$. Changing from $z$ to $w$ is equivalent to rotating by $(UK^{-1})^\top$.

Substituting into the cost function:

$$ z^\top \hat{Q}_{obj}z = w^\top K^{-1}U^\top \hat{Q}_{obj}UK^{-1}w + 2z^\top K^{-1}U^\top N_1x_1^* = w^\top Bw + w^\top b$$

With $w$, $B$, and $b$ defined, the optimization problem becomes

\begin{align}
&& \min~ w^\top Bw + w^\top b \\
s.t. && w_2^\top w_2 &= c
\end{align}

In [ ]:
K = return_K(D)

G_of_w = rotate_quadratic(G_of_z,(U/K)')
Q_of_w = rotate_quadratic(Q_of_z,(U/K)')